In [1]:
import sys
import logging
import time
import random
import numpy as np
from collections import defaultdict, namedtuple
from optparse import OptionParser

# Required libraries
import h5py
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from keras.layers.normalization import BatchNormalization as BN

import IPython.display
import matplotlib.pylab as plt
%matplotlib inline

import sys
sys.path.append('../')
# d-script imports
from data_iters.minibatcher import MiniBatcher
from data_iters.iam_hdf5_iterator import IAM_MiniBatcher
from fielutil import fielnet
import viz_tools.array_to_png as a2p

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled)


## Parameter settings

In [2]:
hdf5_file = '/fileserver/icdar13/benchmarking-processed/author_icdar_be.hdf5'
neural_network_params = '../convnets/fielnet/fielnet.hdf5'
num_authors=100
num_forms_per_author=-1
shingle_dim=(120,120)
use_form=True

batch_size=32
lr = 0.01

## Initialize Classification Minibatcher
### Pat's Code

In [2]:

class StepShingler:
    ShingleSize=namedtuple('ShingleSize', ['rows','cols'])
    
    def __init__(self, img, 
                 hstep, vstep, 
                 shingle_size=ShingleSize(rows=120,cols=120)):
        """ given an image of a document fragment, 
        produce deterministic shingles from that document
        
        Arguments:
            img -- array (numpy array interface)
            hstep -- number of columns to step between shingles
            vstep -- number of rows to step between shingles
            shingle_size -- ROWS, COLS format.
        """
        self.img = img
        self.shingle_size = StepShingler.ShingleSize(*shingle_size)
        self.vstep = vstep
        self.hstep = hstep
        
        # warn if danger of leaving off right or bottom edge
        h_step_space = img.shape[1] - self.shingle_size.cols
        # number of pixels left over on right edge
        h_step_margin = h_step_space % hstep
        if h_step_margin != 0:
            logger = logging.getLogger(__name__)
            logger.debug("Horizontal step/window settings do not divide evenly into image width. "
                        "{0} pixels of information will be lost".format(h_step_margin))
        v_step_space = img.shape[0] - self.shingle_size.rows
        # number of pixels left over on bottom edge
        v_step_margin = v_step_space % vstep
        if v_step_margin != 0:
            logger = logging.getLogger(__name__)
            logger.debug("Vertical step/window settings do not divide evenly into image width. "
                        "{0} pixels of information may be lost".format(v_step_margin))
        
    def __iter__(self):
        for row_i in xrange(0, self.img.shape[0] - self.shingle_size[0], self.vstep):
            for col_j in xrange(0, self.img.shape[1] - self.shingle_size[1], self.hstep):
                logger = logging.getLogger(__name__)
                logger.info("Row {0}, Col {1}".format(row_i, col_j))
                end_col = col_j + self.shingle_size[1]
                end_row = row_i + self.shingle_size[0]
                yield self.img[row_i:end_row, col_j:end_col]
                

def zero_one(x):
    """ Normalize byte-sized integer to 0-1 and flip it so 255 (white)
    maps to 0.0
    """
    x = 1. - (x/255.)
    return x
                
def shingles_4_inference(authors_fragments_set, hstep=90, vstep=90,
                         shingle_size=StepShingler.ShingleSize(rows=120, cols=120),
                         var_threshold=None, postprocess=zero_one,
                         verbose=False):
    """ Given a dictionary-like object (e.g. HDF5 Groups, etc.) of
    author-keyed fragments, iterate over individual authors, fragments,
    and yield pairs of shingle (using deterministic step shingling) and author id.
    
    Arguments:
        authors_fragments_set -- dictionary-like object
        hstep -- horizontal step size in pixels/cols
        vstep -- vertical step size in pixels/rows
        shingle_size -- (rows, cols) tuple indicating size of shingle
            StepShingler.ShingleSize provides a useful namedtuple to avoid 
            parameter order confusion
        var_threshold -- float or None; if given, do not yield any shingle whose
            variance falls below this quantity
        verbose -- if True, will warn on fragments where there are unshingled pixels at the right
            or bottom edge
    """
    
    try:
        if verbose:
            old_level = logging.getLogger(__name__).getEffectiveLevel()
            logging.getLogger(__name__).setLevel(logging.INFO)
        for author_key in authors_fragments_set:
            logger=logging.getLogger(__name__); logger.info("Author: {0}".format(author_key))
            author_fragments = authors_fragments_set[author_key]
            for fragment_key in author_fragments:
                fragment = author_fragments[fragment_key]
                logger.info("Fragment shape: {0}".format(fragment.shape))
                shingle_iterator = StepShingler(fragment, hstep, vstep, shingle_size)
                for shingle in shingle_iterator:
                    if postprocess is not None:
                        shingle=postprocess(shingle)
                    if var_threshold is not None:
                        shingle_var = np.var(shingle)
                        if shingle_var < var_threshold:
                            continue
                    yield shingle, author_key, fragment_key
    finally:
        if verbose:
            logging.getLogger(__name__).setLevel(old_level)

In [6]:
def demo_shingles():
    # get an ICDAR image
    icdar_path = "/fileserver/icdar13/benchmarking-processed/author_icdar_be.hdf5"
    icdar_hdf5 = h5py.File(icdar_path, "r")
    img = icdar_hdf5['001']['001_1.tif']
    
    inf_shingler = StepShingler(img, hstep=90, vstep=120, shingle_size=(120,120))

    plt.figure(figsize=(10, 3))
    for i, shingle in enumerate(inf_shingler):
        plt.subplot(1,9,i+1)
        plt.imshow(shingle,cmap='gray')
        plt.tick_params(
            axis='both',
            which='both',      
            bottom='off',      
            top='off',        
            left='off',
            right='off',
            labelbottom='off',
            labelleft='off') 

        if (i+1) % 9 == 0:
            plt.show()
            break
            
def demo_shingle_iteration():
    # get ICDAR images
    icdar_path = "/fileserver/icdar13/benchmarking-processed/author_icdar_be.hdf5"
    icdar_hdf5 = h5py.File(icdar_path, "r")

    icdar_shingles = shingles_4_inference(icdar_hdf5, var_threshold=0.025, verbose=False)
    if icdar_shingles is not None:
        for icdar_shingle, author_id, fragment_id in icdar_shingles:
            logger=logging.getLogger(__name__)
            logger.info(icdar_shingle.shape)
            #a2p.display_img_array(icdar_shingle)
            plt.imshow(icdar_shingle, cmap='gray')
            plt.show()
            print "Author: {0}, Fragment: {1}".format(author_id, fragment_id)
            shingle_var = np.var(icdar_shingle.reshape(-1))
            print "Variance: {0}".format(shingle_var)
            IPython.display.clear_output(wait=True)
            time.sleep(0.15)
            
#demo_shingles()
demo_shingle_iteration()

KeyboardInterrupt: 

In [ ]:
iam_m = IAM_MiniBatcher(hdf5_file, num_authors, num_forms_per_author, shingle_dim=shingle_dim, default_mode=MiniBatcher.TRAIN, batch_size=batch_size, train_pct=1.0, test_pct=0.0, val_pct = 0.0)

[X_test, Y_test] = iam_m.get_train_batch(batch_size*20)
print "Shape="+str(X_test.shape)+", Max="+str(X_test.max())

plt.clf
plt.subplots(3,3)
plt.tight_layout()
s = np.random.choice(32,9, replace=False)

for i in xrange(9):
    plt.subplot(3,3,i+1)
    plt.imshow(X_test[s[i]])

## Load a neural network model

In [5]:
model=fielnet(neural_network_params, layer='fc7')
print "Compiling model"
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.7, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd)
print "Finished compilation"

print "Getting the first validation batch"
[X_val, Y_val] = iam_m.get_train_batch(batch_size*10)
X_val = np.expand_dims(X_val, 1)
Y_val = to_categorical(Y_val, num_authors)
print "Finished getting "+str(batch_size*10)+" data points"

Compiling model
Finished compilation
Getting the first validation batch
Finished getting 320 data points


In [ ]:
# Unclear if this is needed
from PIL import Image
def randangle(batch):
    newbatch = np.zeros(batch.shape)
    for i,im in enumerate(batch):
        imangle = np.asarray(Image.fromarray(im.squeeze()).rotate(7.5*np.random.randn()))
        newbatch[i]=imangle
    return newbatch

## Neural network feature extraction

In [ ]:
features = np.zeros(  )
for batch_iter in xrange(total_iters):
    [X_train, Y_train] = iam_m.get_train_batch(batch_size*100)
    X_train = np.expand_dims(X_train, 1)
    features[batch_iter] = model.predict(X_train)

## Feature aggregator

## Save Features for Classifier? (Into Train/Test/Validation?)